# Humanos vs Caballos

En los anteriores ejercicios hemos podido ver a las redes neuronales convolucionales en acción y cómo su naturaleza a través de los kernels mejora el rendimiento drásticamente.

Sin embargo, el dataset de Fashion MNIST es un dataset bastante irreal en términos de espacio y de tamaño para tareas de visión artificial reales. Una de las limitaciones más grandes al entrenar redes convolucionales es que no siempre se puede cargar todo el dataset a la memoria RAM. La memoria RAM es un recurso limitado en cualquier servidor, por lo que se debe buscar una forma más eficiente de poder realizar este tipo de tareas.

En este notebook exploraremos el caso con un dataset más cercano a lo que un ingeniero se puede encontrar *en la vida real* y cómo tratar con el mismo,


## El dataset: Humanos vs Caballos
Usaremos un dataset preparado por Laurence Moroney de Tensorflow el cual contiene imágenes sintéticas de caballos y personas. Primero tenemos que descargar tanto el conjunto de entrenamiento como el conjunto de validación o pruebas.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

En la siguiente celda usaremos funciones de la librería os, para acceder a librerías del sistema operativo de tal manera que podamos acceder al sistema de archivos de la máquina virtual que google colab nos ofrece.


In [ ]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

Los contenidos de los archivos comprimidos .zip se extraerán en el directorio base /tmp/horse-or-human, dentro del mismo se tendrán 2 subdirectorios: `horse` y `human`

El **conjunto de entrenamiento** corresponde a los datos que se usan para decirle a la red neuronal 'así se ve un caballo'o 'así se ve una persona'.

Algo que considerar es que no estamos etiquetando explícitamente las imágenes como caballos o personas. Esto se logrará usando un generador de Tensorflow que es capaz de inferir las clases en base a la estructura de directorios de nuestro dataset.

A continuación definimos tales directorios:

In [ ]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_hames = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

## Definiendo el modelo

Primero importamos tensorflow

In [ ]:
import tensorflow as tf

Agregamos las capas convolucionales similar al ejemplo anterior, y *aplanamos* el resultado final para continuar con las capas densamente conectadas.

Seguidamente añadimos las capas densamente conectadas.

Nótese que debido a que usamos un problema de clasificación binaria, la salida de nuestra red neuronal será una función [*sigmoide*](https://wikipedia.org/wiki/Sigmoid_function), de tal manera que la salida ser'a un escalar entre 0 y 1, se puede interpretar este valor como una probabilidad de que la imagen pertenezca a la clase 1.


In [ ]:
model = tf.keras.models.Sequential([
    # La entrada es una imagen de 150 x 150 x 3 canales de color
    # Primera Convolucion
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # segunda convolucion
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # tercera convolucion
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # cuarta
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # quinta
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # capa densamente conectada con 512 unidades
    tf.keras.layers.Dense(512, activation='relu'),
    # una neurona en la salida 1 -> humanos, 0 -> caballos
    tf.keras.layers.Dense(1, activation='sigmoid')
])

El método model.summary() imprime un resumen del modelo

In [ ]:
model.summary()

La columna "output shape" nos muestra cómo evolucionan las dimensiones de los mapas de características en cada capa sucesiva. Las capas convolucionales reducen la dimensión gracias al padding y cada capa de pooling reduce a la mitad las dimensiones.


Ahora, pasaremso a configurar el entrenamiento usando el concepto de *entropía cruzada binaria* o `binary_crossentropy` para la función de costo, debido a que nuestra tarea es una clasificación binaria.

Usaremos el optimizador `rmsprop` con un learning rate de `0,001`. Durante el entrenamiento deseamos monitorear la precisión de clasificación.

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

### Data Preprocessing

Configuraremos generadores de datos que accederán a los directorios donde se encuentran las imágenes, convertirán las mismas a tensores `float32` y alimentarán los mismos a nuestra red con sus correspondientes etiquetas. Los generadores entregarán conjuntos de imágenes de tamaño 300x300 y sus etiquetas.

Como exploramos anteriormente, usualmente se debería normalizar los datos de alguna manera para mejorar el proceso de entrenamiento de la red. En este caso, las imágenes tienen valores de los pixeles que varían entre 0 y 255, procuraremos que este rango esté entre 0 y 1.

En Keras, esta funcionalidad se puede acceder a través de la clase `keras.preprocessing.image.ImageDataGenerator`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalizar imágenes
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# el generador hace 'fluir'imagenes desde un directorio
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # Este es el directorio raíz para entrenamiento
        target_size=(150, 150),  # Se hace un resize
        batch_size=128,
        # definimos la forma de inferir los labels
        class_mode='binary')

# 
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # directorio para validación
        target_size=(150, 150),  # resize
        batch_size=32,
        # 
        class_mode='binary')

### Entrenamiento
Se entrenará la red por 15 épocas. Esto puede tomar algunos minutos en completarse.

Monitoree los valores en cada época.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### Haciendo predicciones

Vamos a usar el modelo entrenado haciendo predicciones. La celda a continuación nos permitirá subir uno o más archivos para realizar predicciones sobre los mismos.

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

### Visualizando Representaciones Internas

Tambien es divertido visualizar qué tipo de filtros y qué tipo de características nuestra red ha aprendido a resaltar.

Seleccionando una imagen aleatoria en el conjunto de entrenamiento, se puede generar una figura con la salida de cada capa convolucional representando a cada filtro y su correspondiente mapa de características, las mismas se suelen llamar **representaciones internas** de la red neuronal.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

Como se puede observar, a medida que se avanza en las capas más profundas, las representaciones son cada vez más abstractas y compactas. Los valores altos representan a los lugares a los cuales la red le está prestando más atención.

Las representaciones contienen cada vez menos información acerca de los pixeles originales de la imagen pero más información refinada acerca de la clase de la imagen. Se puede entender una convnet (o cualquier red neuronal profunda en general) como un pipeline de destilación de la información.